# Код 1

In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c hseds-texts-2020

! mkdir hseds-texts-2020
! unzip train.csv.zip -d hseds-texts-2020
! unzip test.csv.zip -d hseds-texts-2020

Saving kaggle.json to kaggle.json
  0% 0.00/723k [00:00<?, ?B/s]
100% 723k/723k [00:00<00:00, 107MB/s]
  0% 0.00/1.75M [00:00<?, ?B/s]
100% 1.75M/1.75M [00:00<00:00, 119MB/s]
 57% 5.00M/8.78M [00:00<00:00, 33.7MB/s]
100% 8.78M/8.78M [00:00<00:00, 42.9MB/s]
Archive:  train.csv.zip
  inflating: hseds-texts-2020/train.csv  
Archive:  test.csv.zip
  inflating: hseds-texts-2020/test.csv  


In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

tqdm.pandas()

In [2]:
PATH_TO_TRAIN_DATA = 'hseds-texts-2020/train.csv'

In [3]:
df = pd.read_csv(PATH_TO_TRAIN_DATA)
df.head()

,review_id,negative,positive,score
0,00003c6036f30f590c0ac435efb8739b,There were issues with the wifi connection,No Positive,7.1
1,00004d18f186bf2489590dc415876f73,TV not working,No Positive,7.5
2,0000cf900cbb8667fad33a717e9b1cf4,More pillows,Beautiful room Great location Lovely staff,10.0
3,0000df16edf19e7ad9dd8c5cd6f6925e,Very business,Location,5.4
4,00025e1aa3ac32edb496db49e76bbd00,Rooms could do with a bit of a refurbishment ...,Nice breakfast handy for Victoria train stati...,6.7


In [4]:
test_data = pd.read_csv('hseds-texts-2020/test.csv')
test_data.head()

,review_id,negative,positive
0,00026f564b258ad5159aab07c357c4ca,Other than the location everything else was h...,Just the location
1,000278c73da08f4fcb857fcfe4ac6417,No UK TV but this was a minor point as we wer...,Great location very comfortable clean breakfa...
2,000404f843e756fe3b2a477dbefa5bd4,A tiny noisy room VERY deceptively photographed,The breakfast booked the preceding night but ...
3,000a66d32bcf305148d789ac156dd512,Noisy various electrical devices kicking in r...,Great location Nice bathroom
4,000bf1d8c5110701f459ffbedbf0d546,No Negative,Great location and friendly staff


In [5]:
import string

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize


# Добавим стеммер
from nltk.stem import LancasterStemmer
from nltk.stem import PorterStemmer

lancaster = LancasterStemmer()
# porter = PorterStemmer()

# Стоп-слова
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english')) 

# Немножко переопределим функцию
def process_text(text, stem = True, raw = False):
    if stem:
        text = ' '.join([lancaster.stem(word) for word in word_tokenize(text.lower())\
                  if (word not in string.punctuation) and (word not in stop_words)])
    elif raw:
        text = ' '.join([lancaster.stem(word) for word in word_tokenize(text.lower())\
                  if (word not in string.punctuation)])
    else:
        text = ' '.join([word for word in word_tokenize(text.lower())\
                  if (word not in string.punctuation) and (word not in stop_words)])
    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# !pip install bpemb

In [ ]:
"""
from bpemb import BPEmb
bpemb_en = BPEmb(lang="en", vs=100000, dim=300)

bpemb_en.emb.vectors.shape
"""

'\nfrom bpemb import BPEmb\nbpemb_en = BPEmb(lang="en", vs=100000, dim=300)\n\nbpemb_en.emb.vectors.shape\n'

In [ ]:
def dummy_func(x):
    pos = process_text(x['positive'], stem=False, raw = True)
    neg = process_text(x['negative'], stem=False, raw = True)
    # return bpemb_en.encode_ids_with_eos(neg) + bpemb_en.encode_ids(pos)
    return neg + ' <EOS> ' + pos

In [ ]:
df['review'] = df.progress_apply(dummy_func, axis = 1)
test_data['review'] = test_data.progress_apply(dummy_func, axis = 1)

KeyboardInterrupt: ignored

In [ ]:
df['review'][0]

'issu wif connect <sep> posit'

In [ ]:
df['review'] = df['review'].apply(lambda x: x.split())

# Трансформеры

In [ ]:
WORDS = set()
for sent in list(df['review']):
    for w in sent:
        WORDS.add(w)
"""   
for sent in list(df['negative'].apply(lambda x: x.split())):
    for w in sent:
        WORDS.add(w)
"""

int2word = dict(enumerate(tuple(WORDS)))
word2int = {w: ii for ii, w in int2word.items()}

In [ ]:
# MAX_LEN = max(max(df['positive'].apply(len)), max(df['negative'].apply(len)))
# MAX_LEN

In [ ]:
MAX_LEN = max(df['review'].apply(len))
MAX_LEN

341

In [ ]:
train_data, test_data = train_test_split(df)

In [ ]:
from torch.nn.utils.rnn import pad_sequence

#train_pos_pad = pad_sequence([torch.as_tensor([word2int[w] for w in seq][:MAX_LEN]) for seq in df_train['positive']], 
#                           batch_first=True)

X_train = pad_sequence([torch.as_tensor([word2int[w] for w in seq][:MAX_LEN]) for seq in train_data['review']], 
                    batch_first=True)
X_train = F.pad(X_train, (0, MAX_LEN - X_train.shape[1]))
y_train = train_data['score']

X_test = pad_sequence([torch.as_tensor([word2int[w] for w in seq][:MAX_LEN]) for seq in test_data['review']], 
                    batch_first=True)
X_test = F.pad(X_test, (0, MAX_LEN - X_test.shape[1]))

y_test = test_data['score']

X_train.shape, X_test.shape

(torch.Size([75000, 341]), torch.Size([25000, 341]))

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

class TransformerModel(nn.Module):

    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, seqlen, vocab = None, dropout=0.5):
        super(TransformerModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        if vocab:
            self.encoder = nn.Embedding.from_pretrained(torch.as_tensor(vocab.vectors))
        else:
            self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.seqlen = seqlen
        self.decoder = nn.Linear(ninp * seqlen, nhid)
        self.fc = nn.Linear(nhid, 1)

        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (F.pad(torch.ones(self.seqlen, sz), (0, self.seqlen - sz, 0, 0)) == 1) # .transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = src.transpose(0, 1)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = output.transpose(0, 1)
        output = nn.Flatten()(output)
        output = self.decoder(output)
        output = self.fc(F.relu(output))
        return output

  
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        ## TODO
        self._tokens, self.targets  = df[0], df[1]

    def __len__(self):
        ## TODO
        return len(self._tokens)
    
    def __getitem__(self, idx: int):
        ## TODO
        return self._tokens[idx], torch.as_tensor(self.targets[idx], dtype=torch.float32).unsqueeze(0)

In [ ]:
ntokens = len(word2int) # bpemb_en.vectors.shape[0] # the size of vocabulary
emsize = 512 # bpemb_en.vectors.shape[1] # embedding dimension
nhid = 2048 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 8 # the number of heads in the multiheadattention models
dropout = 0.1 # the dropout value
seqlen = MAX_LEN # MAX_LEN
model = TransformerModel(ntokens, emsize, nhead, nhid, nlayers, seqlen, None, dropout)
model.cuda()

TransformerModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=

In [ ]:
BATCH_SIZE = 4

train_dataset = ReviewsDataset((X_train, np.array(y_train)))
test_dataset = ReviewsDataset((X_test, np.array(y_test)))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
criterion = nn.MSELoss()
lr = 1e-5 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    total_loss_MAE = 0.
    start_time = time.time()
    counter = 0
    for batch in train_dataloader:
        inputs, targets = batch
        inputs, targets = inputs.cuda(), targets.cuda()
        # mask = model.generate_square_subsequent_mask((data != 0).sum())
        optimizer.zero_grad()
        output = model(inputs) # mask.cuda())
        loss = criterion(output, targets)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) # no explode
        optimizer.step()

        total_loss += loss.item()
        total_loss_MAE += MAE(output.detach().cpu(), targets.cpu())
        log_interval = 250
        if counter % log_interval == 0 and counter > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.5f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | MAE {:5.2f}'.format(
                    epoch, counter, len(train_dataloader), lr,
                    # scheduler.get_last_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, total_loss_MAE / log_interval ))
            total_loss_MAE = 0
            total_loss = 0
            start_time = time.time()
        counter += 1

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    with torch.no_grad():
        for batch in data_source:
            inputs, targets = batch
            inputs, targets = inputs.cuda(), targets.cuda()
            output = eval_model(inputs)
            total_loss += MAE(output.detach().cpu(), targets.cpu())
    return total_loss / len(data_source)

In [ ]:
best_val_loss = float("inf")
epochs = 10 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, test_dataloader)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f}'
    .format(epoch, (time.time() - epoch_start_time),
    val_loss))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
    
    # scheduler.step()

| epoch   1 |   250/18750 batches | lr 0.00001 | ms/batch 133.33 | loss  2.38 | MAE  1.18
| epoch   1 |   500/18750 batches | lr 0.00001 | ms/batch 132.63 | loss  2.01 | MAE  1.09
| epoch   1 |   750/18750 batches | lr 0.00001 | ms/batch 132.40 | loss  2.10 | MAE  1.09
| epoch   1 |  1000/18750 batches | lr 0.00001 | ms/batch 132.59 | loss  2.13 | MAE  1.11
| epoch   1 |  1250/18750 batches | lr 0.00001 | ms/batch 132.51 | loss  2.06 | MAE  1.10
| epoch   1 |  1500/18750 batches | lr 0.00001 | ms/batch 132.53 | loss  2.18 | MAE  1.11
| epoch   1 |  1750/18750 batches | lr 0.00001 | ms/batch 132.57 | loss  1.94 | MAE  1.07
| epoch   1 |  2000/18750 batches | lr 0.00001 | ms/batch 132.47 | loss  1.93 | MAE  1.07
| epoch   1 |  2250/18750 batches | lr 0.00001 | ms/batch 132.46 | loss  2.26 | MAE  1.13
| epoch   1 |  2500/18750 batches | lr 0.00001 | ms/batch 132.48 | loss  1.90 | MAE  1.05
| epoch   1 |  2750/18750 batches | lr 0.00001 | ms/batch 132.53 | loss  2.09 | MAE  1.12
| epoch   

KeyboardInterrupt: ignored

Собственно написанный трансформер не выбивает менее 1 MAE ((

In [ ]:
evaluate(model, test_dataloader)

# Pretrained из transformers

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 17.4MB/s 
     |████████████████████████████████| 2.9MB 48.8MB/s 
     |████████████████████████████████| 890kB 47.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=8de011697ec3dc948080bbfef8d1053dbab50de6841f2890947548b361b256d9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
F.pad(model(**tokenizer(["Hello, my dog is cute"], return_tensors="pt"))['last_hidden_state'], (0, 0, 0, MAX_LEN - 6, 0, 0)).shape

In [ ]:
# df['review'] = df.progress_apply(lambda x: tokenizer(x['positive'])['input_ids'] + tokenizer(x['negative'])['input_ids'], axis=1)
df['review'] = df.progress_apply(lambda x: x['positive'] + ' <|endoftext|> ' + x['negative'], axis=1)

In [ ]:
train_data, test_data = train_test_split(df)

In [ ]:
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        ## TODO
        self._tokens = np.array(df[0])
        self._targets = np.array(df[1])

    def __len__(self):
        ## TODO
        return len(self._tokens)
    
    def __getitem__(self, idx: int):
        ## TODO
        return self._tokens[idx], torch.as_tensor(self._targets[idx], dtype=torch.float32).unsqueeze(0)

In [ ]:
BATCH_SIZE = 2

train_dataset = ReviewsDataset((train_data['review'], np.array(train_data['score'])))
test_dataset = ReviewsDataset((test_data['review'], np.array(train_data['score'])))

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
class MyGPT(nn.Module):

    def __init__(self, seqlen):
        super(MyGPT, self).__init__()
        from transformers import GPT2Model, GPT2Config, GPT2Tokenizer
        self.seqlen = seqlen
        self.configuration = GPT2Config()
        self.n_embd = self.configuration.n_embd
        self.GPT = GPT2Model(self.configuration)
        self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        self.tokenizer.pad_token = self.tokenizer.eos_token
        # tokenizer(["Hello, my dog is cute", "Hello, cute"], return_tensors="pt", max_length=MAX_LEN, truncation=True, padding=True)
        self.fc1 = nn.Linear(self.n_embd * self.seqlen, 1)
        # self.fc2 = nn.Linear(self.seqlen, 1)
        

    def forward(self, src):
        src = self.tokenizer(src, return_tensors="pt", max_length=self.seqlen, truncation=True, padding=True)
        for k in src:
            src[k] = src[k].cuda()
        out = self.GPT(**src)['last_hidden_state']
        out = F.pad(out, (0, 0, 0, self.seqlen - out.shape[1], 0, 0))
        out = nn.Flatten()(out)
        # out = F.relu(self.fc1(out).view(-1, self.seqlen)) # F.relu(self.fc1(out))
        # out = self.fc2(out)
        out = self.fc1(out)

        return out

In [ ]:
model = MyGPT(1024)
model.cuda()

MyGPT(
  (GPT): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=

In [ ]:
criterion = nn.MSELoss()
lr = 1e-4 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)
# optimizer = torch.optim.Adam(model.parameters(), lr=lr)

import time
def train():
    model.train() # Turn on the train mode
    total_loss = 0.
    total_loss_MAE = 0.
    start_time = time.time()
    counter = 0
    for batch in train_dataloader:
        inputs, targets = batch
        targets = targets.cuda()
        # inputs, targets = inputs.cuda(), targets.cuda()
        # mask = model.generate_square_subsequent_mask((data != 0).sum())
        optimizer.zero_grad()
        output = model(inputs) # mask.cuda())
        loss = criterion(output, targets)
        loss.backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) # no explode
        optimizer.step()

        total_loss += loss.item()
        total_loss_MAE += MAE(output.detach().cpu(), targets.cpu())
        log_interval = 250
        if counter % log_interval == 0 and counter > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.5f} | ms/batch {:5.2f} | '
                  'loss {:5.2f} | MAE {:5.2f}'.format(
                    epoch, counter, len(train_dataloader), lr,
                    # scheduler.get_last_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, total_loss_MAE / log_interval ))
            total_loss_MAE = 0
            total_loss = 0
            start_time = time.time()
        counter += 1

def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    with torch.no_grad():
        for batch in data_source:
            inputs, targets = batch
            inputs, targets = inputs.cuda(), targets.cuda()
            output = eval_model(inputs)
            total_loss += MAE(output.detach().cpu(), targets.cpu())
    return total_loss / len(data_source)

In [ ]:
best_val_loss = float("inf")
epochs = 10 # The number of epochs
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train()
    val_loss = evaluate(model, test_dataloader)
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f}'
    .format(epoch, (time.time() - epoch_start_time),
    val_loss))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
    
    # scheduler.step()

ValueError: ignored

# Simple Transformers

Как transformers, но по проще)

In [7]:
df['review'] = df.progress_apply(lambda x: x['positive'] + ' ' + x['negative'], axis=1)
test_data['review'] = test_data.progress_apply(lambda x: x['positive'] + ' ' + x['negative'], axis=1)

# train_data, test_data = train_test_split(df)

class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        ## TODO
        self._tokens = np.array(df[0])
        self._targets = np.array(df[1])

    def __len__(self):
        ## TODO
        return len(self._tokens)
    
    def __getitem__(self, idx: int):
        ## TODO
        return self._tokens[idx], torch.as_tensor(self._targets[idx], dtype=torch.float32).unsqueeze(0)


# BATCH_SIZE = 2

# train_dataset = ReviewsDataset((train_data['review'], np.array(train_data['score'])))
# test_dataset = ReviewsDataset((test_data['review'], np.array(train_data['score'])))

# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [8]:
!pip install simpletransformers

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Preparing train data
train_df = df[['review', 'score']]
train_df.columns = ["text", "labels"]

# Preparing eval data
eval_df = df[['review', 'score']]
eval_df.columns = ["text", "labels"]

# Enabling regression
# Setting optional model configuration
model_args = ClassificationArgs()
model_args.num_train_epochs = 100
model_args.regression = True

# Create a ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args=model_args
)

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

# Make predictions with the model
# predictions, raw_outputs = model.predict(["Sam was a Wizard"])

INFO:filelock:Lock 140153157317744 acquired on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

INFO:filelock:Lock 140153157317744 released on /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b.lock
INFO:filelock:Lock 140151697214264 acquired on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock


Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

INFO:filelock:Lock 140151697214264 released on /root/.cache/huggingface/transformers/51ba668f7ff34e7cdfa9561e8361747738113878850a7d717dbc69de8683aaad.c7efaa30a0d80b2958b876969faa180e485944a849deee4ad482332de65365a7.lock
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of R

...

INFO:filelock:Lock 140151697215328 released on /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


  0%|          | 0/100000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Running Epoch 0 of 100:   0%|          | 0/12500 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7ff81dfeef60>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


INFO:simpletransformers.classification.classification_model:{'eval_loss': 1.007052994978428}


In [ ]:
# Результат после 2х эпох
MAE(model_outputs, test_data['score'])

0.7252751875

In [ ]:
# Clear output
!rm -r outputs

In [ ]:
model.train_model(train_df)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


KeyboardInterrupt: ignored

Без предварительной обработки текста (со стандартной токенизацией из библиотеки). RoBERTa набирает 0.72 MAE за 2 эпохи обучения

In [ ]:
kaggle_df = pd.read_csv('hseds-texts-2020/test.csv')
kaggle_df['review'] = kaggle_df.progress_apply(lambda x: x['positive'] + ' ' + x['negative'], axis=1)
kaggle_data = kaggle_df['review']
kaggle_data.columns = ["text"]

kaggle_data.head()

0     Just the location   Other than the location e...
1     Great location very comfortable clean breakfa...
2     The breakfast booked the preceding night but ...
3     Great location Nice bathroom  Noisy various e...
4        Great location and friendly staff No Negative
Name: review, dtype: object

In [ ]:
predictions, raw_outputs = model.predict(kaggle_data)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


In [ ]:
pred = np.clip(raw_outputs, 0, 10)
pred = pd.concat([kaggle_df['review_id'], pd.Series(pred)], axis = 1)
pred.columns = ['review_id', 'score']
pred.head()

,review_id,score
0,00026f564b258ad5159aab07c357c4ca,3.800781
1,000278c73da08f4fcb857fcfe4ac6417,9.898438
2,000404f843e756fe3b2a477dbefa5bd4,6.585938
3,000a66d32bcf305148d789ac156dd512,5.804688
4,000bf1d8c5110701f459ffbedbf0d546,9.718750


In [ ]:
pred.to_csv("submission.csv", index=False)